In [ ]:
import copy
import itertools
import os
import sys

import numpy as np

from hera_cal.io import HERACal, HERAData 

In [ ]:
simpleredcal_path = '/lustre/aoc/projects/hera/mmolnar/simpleredcal'
sys.path.insert(1, simpleredcal_path)

from align_utils import idr2_jdsx

In [ ]:
jd_label_dict = np.load(os.path.join(simpleredcal_path, 'b2f2_jd_dict.npz'), \
                        allow_pickle=True)['jd_dict'].item()

h1c_idr2_dir = '/lustre/aoc/projects/hera/H1C_IDR2/IDR2_2/'
new_idr2_dir = '/lustre/aoc/projects/hera/mmolnar/LST_bin/H1C_IDR2'

In [ ]:
# # Creating new folders
# for jd in idr2_jdsx:
#     if not os.path.exists(os.path.join(new_idr2_dir, str(jd))):
#         os.mkdir(os.path.join(new_idr2_dir, str(jd)))

In [ ]:
cal_jds = list(set(itertools.chain(*list(itertools.chain(*jd_label_dict.values())))))
cal_jds.extend([str(i) for i in jd_label_dict.keys()])
cal_jds = sorted(cal_jds)

In [ ]:
# # Copying smooth_abs.calfits files
# from shutil import copy2

# for cal_jd in cal_jds:
#     src = os.path.join(h1c_idr2_dir, str(int(float(cal_jd))), 'zen.{}.HH.smooth_abs.calfits'.format(cal_jd))
#     dst = os.path.join(new_idr2_dir, str(int(float(cal_jd))), 'zen.{}.HH.smooth_abs.calfits'.format(cal_jd))
#     if not os.path.exists(dst):
#         copy2(src, dst)

In [ ]:
tot_flags_fn = 'b2f2_tot_flags.npz'
tot_flags = np.load(os.path.join(simpleredcal_path, tot_flags_fn))['flags']

In [ ]:
modz_fn = 'b2f2_mz_flags.npz'
modz_flags = np.load(os.path.join(simpleredcal_path, modz_fn))['simga5']

In [ ]:
tot_flags.shape

In [ ]:
# Band 2 Frequencies
from red_utils import calfits_to_flags, find_zen_file
jdt_ref = 2458098.43869
hd = HERAData(find_zen_file(jdt_ref))
b2_freq_start = 150.3*1e6 # MHz
b2_freq_stop = 167.8*1e6 # MHz
band2_chans = np.where(np.logical_and(hd.freqs >= b2_freq_start, hd.freqs <= b2_freq_stop))[0]

calf = calfits_to_flags(jdt_ref, 'smooth_abs', pol='ee', add_bad_ants=None)[band2_chans, ...]

In [ ]:
jdt_idx = list(jd_label_dict.keys()).index(jdt_ref)
tot_flags[0, :, jdt_idx*60:(jdt_idx+1)*60, :].sum()

In [ ]:
calf.sum()

In [ ]:
cal_jd = cal_jds[0]
cal_fn = os.path.join(h1c_idr2_dir, str(int(float(cal_jd))), 'zen.{}.HH.smooth_abs.calfits'.format(cal_jd))
jd_idx = idr2_jdsx.index(int(float(cal_jd)))

In [ ]:
hc = HERACal(cal_fn)
sc_gains, sc_flags, sc_quals, sc_tot_qual = hc.read()

In [ ]:
sc_flags

In [ ]:
new_flags = copy.deepcopy(sc_flags)